# Get Security Recommendations

    Author: Jing Nghik
    Last Updated 2/9/2021

Purpose: This notebook is an example to get vulnerabilities

## Requirements: 

### Scopes Required
- SecurityRecommendation.Read
- SecurityRecommendation.Read.All

### A client AppID/Secret or Device Code Authentication
[Proper MDATP API Token](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-webapp)
        
    Microsoft Defender ATP exposes much of its data and actions through a set of programmatic APIs. Those APIs will help you automate work flows and innovate based on Microsoft Defender ATP capabilities.

## Set Variables and Import required modules and credentials 

In [1]:
## Change these if required
$resource = "https://api.securitycenter.windows.com" ## resource Dont change since we want to query MDATP REST API Resource
$grant_type = "client_credentials" ## This is using a device_code flow (supports: client_credentials, device_code)

## Dont really need to change these unless necessary
$VerbosePreference = "ignore"   ## Set to 'continue to get verbose output'
## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"; . $scriptPath  # this will run the script

The below script needs to be able to find the current output cell; this is an easy method to get it.


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

## Authenticate with Graph API and get access Token for MDATP Resource

In [2]:
## Get a token if authenticates properly. 

$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = $resource  ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = $grant_type  ## This is using a device_code flow (supports: client_credentials, device_code)
    #scope = "https://api.securitycenter.microsoft.com/Machine.ReadWrite.All" ## set the scope
};

## this will authenticate with Graph API and token will be stored in <tokenResponse> variable.
$token = (Invoke-GraphAuthentication -authParams $authParams)

## This is the authentication header for REST API Call
if ($token.access_token -ne $null) {
    $authHeader = @{ 
        'Content-Type' = 'application/json'
        Accept = 'application/json'
        Authorization = "Bearer $($token.access_token)" 
    }
}; $authHeader


Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/e4194ef7-b5d7-4db1-b7e9-a73af406f276/oauth2/token


Received Token!
Connected and Access Token received and will expire 1612913255

Name                           Value
----                           -----
Content-Type                   application/json
Accept                         application/json
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyIsImtpZCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyJ9.eyJhdWQiOiJodHRwczovL2FwaS5…



## Gets top 10 security recommendations

In [4]:
## Set the path you want to query
$endpoint = "vulnerabilities?top=10"
$method = "GET"

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/api/$endpoint"; Write-Host "URI: $uri"
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -ErrorAction Stop)

## Filter results only for these columns
$Response = ($Response.value | Select-Object @('id','name','description','severity','exposedMachines'))

## Grab first 10 and convert results to table format
$Response | Select -first 10 | FT

URI: https://api.securitycenter.windows.com/api/vulnerabilities?top=10

id             name           description                                                        severity exposedMachines
--             ----           -----------                                                        -------- ---------------
CVE-2021-24074 CVE-2021-24074 Windows TCP/IP Remote Code Execution Vulnerability                 Critical               1
CVE-2021-1692  CVE-2021-1692  Hyper-V Denial of Service Vulnerability                            High                   1
CVE-2021-1732  CVE-2021-1732  Windows Win32k Elevation of Privilege Vulnerability                High                   1
CVE-2021-1734  CVE-2021-1734  Windows Remote Procedure Call Information Disclosure Vulnerability High                   1
CVE-2021-1731  CVE-2021-1731  PFX Encryption Security Feature Bypass Vulnerability               Medium                 1
CVE-2021-1698  CVE-2021-1698  Windows Win32k Elevation of Privilege Vulner